In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
#gpt4omini, llama317B
generation_model = 'llama317B'

In [ ]:
import os
from openai import OpenAI
from pydantic import BaseModel
import json

In [ ]:
with open('../Datasets/all_prompts_task1.json', 'r') as file:
    user_prompt = file.read()

In [ ]:
if(generation_model == 'gpt4omini'):
    client = OpenAI(
        api_key = os.getenv('OPEN_AI_API')
    )

    def create_travel_plan(system_prompt, user_prompt):
        chat_completion = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=[
                {"role": "system","content": system_prompt,},
                {"role": "user","content": user_prompt,}
            ]
        )

        return chat_completion.choices[0].message.content


    for filename in os.listdir('Prompts/prompts_routesOP'):
        index = filename[:-4][7:]
        with open(f'Prompts/prompts_routesOP/{filename}', 'r') as file:
            system_prompt = file.read()
            plan = create_travel_plan(system_prompt, user_prompt)
            with open(f'Outputs/plans/Plan_{index}.txt', 'w') as f:
                f.write(plan)

In [ ]:
from huggingface_hub import login
login()

In [ ]:
torch.cuda.get_device_name(device=None)

In [ ]:
if(generation_model == 'llama317B'):
    import transformers
    import torch
    
    model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device="cuda",
    )
    
    for filename in os.listdir('Prompts/prompts_routesOP'):
        
        #testing purpose
        #if(filename == 'Prompt_1.txt'):
        #    continue

        index = filename[:-4][7:]
        with open(f'Prompts/prompts_routesOP/{filename}', 'r') as file:
            system_prompt = file.read()
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]

        outputs = pipeline(
            messages,
            max_new_tokens=5000, # the llama 3.1 won't increase the detail with more token limit, neither decrese detail with less token, so just create a token limit that won't cut off the text.
            do_sample = False
        )

        with open(f'Outputs/plans/Llama317B/Plan_{index}.txt', 'w') as f:
                    f.write(outputs[0]["generated_text"][-1]['content'])